# Limpieza de datos y preprocesado de textos

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import unicodedata,string

PATH="/home/enriqueotero/datasets/nlp/corpusCriticasCine/"

def clean(x):
   x = unicodedata.normalize('NFKD', x).encode('ascii','ignore').lower()
   replace_punctuation = string.maketrans(string.punctuation, ' '*len(string.punctuation))
   x = x.translate(replace_punctuation)
   x = re.sub('@%$&[\n/:!,;)()_?¿¡<>]', ' ', x)
   x = re.sub(' - ', ' ', x)
   x = re.sub(' +',' ', x).strip()
   return x

def parseFile(filename):
    
    try:
        data = open(filename,'r').read()
        soup = BeautifulSoup(data, "html.parser")  
        review_text = soup.body.get_text()
        letters_only = clean(review_text)
        #letters_only = re.sub("[^a-zA-Z]", " ", review_text) 

        result = letters_only

    except:
        result = ""
        
    return result

parseFile(PATH + str(999) + ".xml")

'bajo mi modesta opinion me resulta una pelicula casi documental a cerca de los acontecimientos sucedidos en la casa real inglesa a tenor de la muerte de lady di y muy especialmente visto desde el punto de vista de la riena isabel de inglaterra la pelicula es entretenida e interesante sobre todo por lo reciente de los acontecimientos pero cuando termino mi sensacion fue mas la de una intencion por parte del creador de la pelicula de reconciliar la opinion publica con la imagen de la monarquia francamente lo tomare como una vision romantica de lo que pudo pasar por la mente de la monarca tras la muerte de lady di y no como lo que realmente sucediera en cualquier caso me quedo con la excepcional interpretacion de helen mirren actriz que me era completamente desconocida hasta esta pelicula asi como el parecido fisico de la reina es asombroso queda muy lejos el reflejo del principe carlos en el actor que lo interpreta alex jennings de espaldas si que da el pego pero visto de cara creo que 

In [2]:
N_DOCS = 1000
mydict = {i:parseFile(PATH+str(i)+".xml") for i in range(N_DOCS)}
df = pd.DataFrame(data=mydict.items(), columns=['id', 'text'])
df = df[df["text"] != ""].dropna()
df.head()
df.to_csv(PATH+"clean_reviews.csv", index=False)

# Representación Bag of Words con TF-IDF

In [3]:
#corpus = ['This is the first document.','This is the second second document.','And the third one.','Is this the first document?']
corpus = df["text"].tolist()

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer=u'word', use_idf=True, max_df=0.3, lowercase=False)
#vectorizer = TfidfVectorizer(analyzer=u'word', use_idf=True)
tfidf_matrix = vectorizer.fit_transform(corpus)
features = vectorizer.get_feature_names()

# Extracción de palabras clave

In [5]:
import operator
import numpy as np

def getKeywords(myvector, top_n, features):
    myweights = myvector.toarray().flatten()
    indices = np.argsort(myweights)[::-1]
    top_features = [features[i] for i in indices[:top_n]]
    return top_features 

N_KEYWORDS = 10
def keywords(myid):
    return getKeywords(tfidf_matrix[myid:myid+1], N_KEYWORDS, features)

print keywords(1)

[u'jedi', u'jedis', u'rollo', u'anaquino', u'palpatino', u'katana', u'jedai', u'samurais', u'peinado', u'batallitas']


In [6]:
df["keywords"] = map(keywords, range(len(df)))

In [7]:
df.head()

,id,text,keywords
2,2,cada vez me gusta menos el cine de masas las p...,"[peli, puta, chorradas, cruise, vieja, huir, o..."
3,3,el otro dia fui a ver la de los sioux como dec...,"[jedi, jedis, rollo, anaquino, palpatino, kata..."
4,4,es que no la cagan en ninguna todas las pelis ...,"[pelis, yankis, madre, repetirse, pa, hacen, c..."
5,5,es un dolor esto del cine yo ya voy con miedo ...,"[spiderman, tele, peli, puta, mico, pastora, v..."
6,6,tras una insufrible primera media hora la casa...,"[cera, acojonante, peli, risa, gore, casa, com..."


# Clustering

In [13]:
from sklearn.cluster import KMeans

num_clusters = 100
clustering = KMeans(n_clusters=num_clusters)
%time clustering.fit(tfidf_matrix)

clusters = clustering.labels_.tolist()

CPU times: user 16.7 s, sys: 36 ms, total: 16.7 s
Wall time: 16.7 s


In [14]:
df["cluster"] = clusters
df.head()

,id,text,keywords,cluster
2,2,cada vez me gusta menos el cine de masas las p...,"[peli, puta, chorradas, cruise, vieja, huir, o...",44
3,3,el otro dia fui a ver la de los sioux como dec...,"[jedi, jedis, rollo, anaquino, palpatino, kata...",64
4,4,es que no la cagan en ninguna todas las pelis ...,"[pelis, yankis, madre, repetirse, pa, hacen, c...",44
5,5,es un dolor esto del cine yo ya voy con miedo ...,"[spiderman, tele, peli, puta, mico, pastora, v...",44
6,6,tras una insufrible primera media hora la casa...,"[cera, acojonante, peli, risa, gore, casa, com...",64


In [15]:
df[df["cluster"] == 4]

,id,text,keywords,cluster
153,153,el argumento de 16 calles es impecable para te...,"[muchacho, detective, pregunta, delincuente, w...",4
402,402,en los anos ochenta fueron pocos los que se re...,"[jonson, don, policias, motivaron, aclarar, fa...",4
408,408,hubiese preferido que michael mann artifice de...,"[miami, mann, farrel, forrados, corrupcion, na...",4
409,409,la esperada y deseada adaptacion al cine de la...,"[vice, tosar, miami, policias, tubbs, sonny, e...",4
417,417,desde que el atraco protagonizado por de niro ...,"[miami, mann, vice, sonny, collateral, corrupc...",4
426,426,esta ola de revivalismo que nos recorre tiene ...,"[miami, vice, caja, mann, tiros, lanchas, pand...",4
434,434,dirigida por michael mann esa simple frase ya ...,"[mann, miami, corrupcion, sonny, farrell, foxx...",4
439,439,corrupcion en miami empieza con el final de ot...,"[coopera, nash, miami, mann, traficantes, estr...",4
447,447,hace poco que leia una noticia en la que una w...,"[miami, vice, tosar, tecnicamente, mann, luis,...",4
491,491,michael mann se encarga de llevar a la gran pa...,"[mann, sonny, collateral, miami, crockett, int...",4


In [17]:
df[df["cluster"] == 0]

,id,text,keywords,cluster
357,357,en los creditos finales el film es dedicado a ...,"[avion, greengrass, secuestrados, vuelo, pasaj...",0
360,360,cinco anos desde el 11 de septiembre aquella f...,"[aquel, echos, 93, victimas, aviones, greengra...",0
364,364,la maquinaria de hollywood no tiene piedad con...,"[greengrass, americano, trade, center, 93, uni...",0
372,372,el cine de spielberg representa justo lo contr...,"[elegido, autor, greengrass, sabe, spielberg, ...",0
387,387,cuando termina la pelicula uno se queda callad...,"[perdimos, greengrass, cinta, 11, vaya, sienes...",0
394,394,suele ocurrir que cuando alguien pone las expe...,"[avion, interior, lleves, cubra, forzosa, te, ...",0
403,403,es de una dificultad extrema recuperar a trave...,"[93, united, cnn, greengrass, objetivode, clin...",0
406,406,era necesario mucho coraje para abordar aconte...,"[93, sucedio, greengrass, united, terroristas,...",0
428,428,hoy hace 5 anos de los terribles acontecimient...,"[93, avion, despegue, united, mando, norad, ca...",0
429,429,estamos en los dias en los que se conmemora el...,"[greengrass, pasajeros, tripulantes, 11, avion...",0
